## Try Different Ensemble Methods

In [2]:
import warnings
warnings.filterwarnings('ignore')
%run Functions.py

In [17]:
train = pd.read_csv('data/train_c.csv')
test = pd.read_csv('data/test_c.csv')

In [22]:
train,test = get_additional_features(train,test)
train = train.sample(frac=1,random_state=42)

In [23]:
col = test.columns

## Different Sklearn Model Fitting

## GBDT

In [20]:
from sklearn.ensemble import GradientBoostingRegressor

In [37]:
gbdt = GradientBoostingRegressor(n_estimators=600,subsample=0.95,max_features=1,max_depth=3,random_state=42)

In [38]:
gbdt_train,gbdt_test,ID = get_sklearn_stack_data(gbdt,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.547611772746
RMSE of Fold 1 is 8.39131695671
Training 2 Fold
R2 Scored of Fold 2 is 0.413707471482
RMSE of Fold 2 is 10.5053750353
Training 3 Fold
R2 Scored of Fold 3 is 0.559258845243
RMSE of Fold 3 is 8.25738846358
Training 4 Fold
R2 Scored of Fold 4 is 0.592731204536
RMSE of Fold 4 is 7.77134013852
Training 5 Fold
R2 Scored of Fold 5 is 0.504996471476
RMSE of Fold 5 is 8.76344571285
Start Training
Calculating In-Bag R2 Score
0.633373286069
Calculating Out-Bag R2 Score
0.523661153096
Calculating In-Bag RMSE
7.6764162751
Calculating Out-Bag RMSE
8.73777326139


## Lasso

In [46]:
from sklearn.linear_model import Lasso
las = Lasso(alpha=0.01)

In [47]:
las_train,las_test,ID = get_sklearn_stack_data(las,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.584286468708
RMSE of Fold 1 is 8.0439906797
Training 2 Fold
R2 Scored of Fold 2 is 0.428075052594
RMSE of Fold 2 is 10.3758551836
Training 3 Fold
R2 Scored of Fold 3 is 0.57672867781
RMSE of Fold 3 is 8.09208313673
Training 4 Fold
R2 Scored of Fold 4 is 0.591665270971
RMSE of Fold 4 is 7.7815033517
Training 5 Fold
R2 Scored of Fold 5 is 0.583087487658
RMSE of Fold 5 is 8.04253978538
Start Training
Calculating In-Bag R2 Score
0.577591474995
Calculating Out-Bag R2 Score
0.552768591548
Calculating In-Bag RMSE
8.2397265984
Calculating Out-Bag RMSE
8.46719442742


## Random Forest

In [73]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=800,max_features=0.95,max_depth=4,n_jobs=-1,random_state=42)

In [74]:
rf_train,rf_test,ID = get_sklearn_stack_data(rf,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.587421600232
RMSE of Fold 1 is 8.01360112699
Training 2 Fold
R2 Scored of Fold 2 is 0.438367713908
RMSE of Fold 2 is 10.282066635
Training 3 Fold
R2 Scored of Fold 3 is 0.595705131447
RMSE of Fold 3 is 7.90860760733
Training 4 Fold
R2 Scored of Fold 4 is 0.609490453225
RMSE of Fold 4 is 7.60976380922
Training 5 Fold
R2 Scored of Fold 5 is 0.598060284617
RMSE of Fold 5 is 7.89680135892
Start Training
Calculating In-Bag R2 Score
0.596541124306
Calculating Out-Bag R2 Score
0.565809036686
Calculating In-Bag RMSE
8.05278494973
Calculating Out-Bag RMSE
8.34216810749


## Light GBM

In [84]:
params = params = {
            'objective': 'regression',
            'metric': 'rmse',
            'boosting': 'gbdt',
            'learning_rate': 0.0045 , #small learn rate, large number of iterations
            'verbose': 0,
            'num_iterations': 500,
            'bagging_fraction': 0.95,
            'bagging_freq': 1,
            'bagging_seed': 42,
            'feature_fraction': 0.95,
            'feature_fraction_seed': 42,
            'max_bin': 100,
            'max_depth': 3,
            'num_rounds': 800
        }

In [85]:
lgb_train, lgb_test,ID = get_lgb_stack_data(params,800,train,col,train['y'],test)

Training 1 Fold
R2 Scored of Fold 1 is 0.601814133337
RMSE of Fold 1 is 7.87258573237
Training 2 Fold
R2 Scored of Fold 2 is 0.442328279141
RMSE of Fold 2 is 10.2457485279
Training 3 Fold
R2 Scored of Fold 3 is 0.604293752034
RMSE of Fold 3 is 7.82415359369
Training 4 Fold
R2 Scored of Fold 4 is 0.619551074047
RMSE of Fold 4 is 7.51109977137
Training 5 Fold
R2 Scored of Fold 5 is 0.595608359975
RMSE of Fold 5 is 7.92085088938
Start Training
Calculating In-Bag R2 Score
0.586607721176
Calculating Out-Bag R2 Score
0.572719119707
Calculating In-Bag RMSE
8.15131440739
Calculating Out-Bag RMSE
8.27488770294


## Stacking those four models, with xgb

In [89]:
train_stack = pd.DataFrame()
train_stack['ID'] = gbdt_train['ID']
train_stack['y'] = gbdt_train['label']
train_stack['model1'] = gbdt_train['predicted']
train_stack['model2'] = las_train['predicted']
train_stack['model3'] = rf_train['predicted']
train_stack['model4'] = lgb_train['predicted']

In [91]:
test_stack = pd.DataFrame()
test_stack['model1'] = gbdt_test['y']
test_stack['model2'] = las_test['y']
test_stack['model3'] = rf_test['y']
test_stack['model4'] = lgb_test['y']

In [94]:
y_mean = np.mean(train.y)

In [103]:
params = {
    'eta': 0.005,
    'max_depth': 2,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}

In [104]:
y_mean

100.66931812782133

In [105]:
col = test_stack.columns
dtrain = xgb.DMatrix(train_stack[col], train_stack['y'])
dtest = xgb.DMatrix(test_stack[col])

In [106]:
xgb_cvalid = xgb.cv(params, dtrain, num_boost_round=2000, early_stopping_rounds=20,
    verbose_eval=50, show_stdv=True,seed=42)
xgb_cvalid[['train-rmse-mean', 'test-rmse-mean']].plot()
print('Performance does not improve from '+str(len(xgb_cvalid))+' rounds')

[0]	train-rmse:12.6383+0.305925	test-rmse:12.6279+0.598657
[50]	train-rmse:11.1425+0.329333	test-rmse:11.1328+0.669345
[100]	train-rmse:10.1269+0.35097	test-rmse:10.1193+0.722135
[150]	train-rmse:9.45305+0.36999	test-rmse:9.44951+0.758414
[200]	train-rmse:9.01564+0.383783	test-rmse:9.02251+0.784844
[250]	train-rmse:8.73291+0.395121	test-rmse:8.75607+0.801707
[300]	train-rmse:8.5501+0.404035	test-rmse:8.58889+0.811879
[350]	train-rmse:8.43014+0.411723	test-rmse:8.48214+0.816475
[400]	train-rmse:8.35182+0.416767	test-rmse:8.41466+0.817739
[450]	train-rmse:8.3002+0.420466	test-rmse:8.3725+0.817775
[500]	train-rmse:8.26508+0.423153	test-rmse:8.34653+0.817391
[550]	train-rmse:8.23951+0.425676	test-rmse:8.33139+0.816391
[600]	train-rmse:8.21889+0.427553	test-rmse:8.32343+0.815737
[650]	train-rmse:8.20132+0.427417	test-rmse:8.31981+0.815773
[700]	train-rmse:8.18681+0.428288	test-rmse:8.31748+0.815108
[750]	train-rmse:8.17382+0.429602	test-rmse:8.31581+0.814357
[800]	train-rmse:8.16039+0.43083

In [108]:
test_stack['ID'] = test['ID']

In [109]:
stack_four_train,stack_four_test, ID = get_xgb_stack_data(params,870,train_stack,col,train_stack['y'],test_stack)

Training 1 Fold
R2 Scored of Fold 1 is 0.599274966754
RMSE of Fold 1 is 7.89764694316
Training 2 Fold
R2 Scored of Fold 2 is 0.420784811507
RMSE of Fold 2 is 10.4417755106
Training 3 Fold
R2 Scored of Fold 3 is 0.599439203767
RMSE of Fold 3 is 7.87200088823
Training 4 Fold
R2 Scored of Fold 4 is 0.619612686662
RMSE of Fold 4 is 7.51049154618
Training 5 Fold
R2 Scored of Fold 5 is 0.593011971561
RMSE of Fold 5 is 7.9462380373
Start Training
Calculating In-Bag R2 Score
0.579361659482
Calculating Out-Bag R2 Score
0.56642472805
Calculating In-Bag RMSE
8.22244330559
Calculating Out-Bag RMSE
8.33363058509


In [110]:
save_results(stack_four_train,stack_four_test,'Stacking_Four_Models_With_xgb.csv')

## Average Yesterday's stacking and LB's result

In [111]:
stack_three_train,stack_three_test = read_data('subStacking_gbdt_lgbm_lasso_with_xgb_rounds_777.csv')

In [114]:
best_current = pd.read_csv('submission/Repro_Ker_2.csv')

In [115]:
Average = 0.75*best_current['y'] + 0.25*stack_three_test['y']

In [116]:
sub = pd.DataFrame({'ID':test['ID'],'y':Average})

In [117]:
sub.to_csv('submission/Average_Sklearnstacking_Best.csv',index=False)

## Average Stacking With Magic xgb and LB's result

In [118]:
average = pd.read_csv('submission/weighted_Stacking_bestxgb_Stacking_0.25.csv')

In [119]:
Average = 0.75*best_current['y'] + 0.25*average['y']

In [120]:
sub = pd.DataFrame({'ID':test['ID'],'y':Average})

In [121]:
sub.to_csv('submission/Average_LB_And_Current_Average.csv',index=False)

## How about averaging LB's result and my sklearn stacked model?

In [4]:
LB_result = pd.read_csv('submission/stacked-models.csv')

In [5]:
stack_three_train,stack_three_test = read_data('subStacking_gbdt_lgbm_lasso_with_xgb_rounds_777.csv')

In [6]:
Average = 0.75*LB_result['y'] + 0.25*stack_three_test['y']

In [8]:
sub = pd.DataFrame({'ID':stack_three_test['ID'],'y':Average})

In [9]:
sub.to_csv('submission/Average_LB_Sklearn_Stacking.csv',index=False)